<a href="https://colab.research.google.com/github/hongqin/Python-CoLab-bootcamp/blob/master/simple_reinforcement_learning_GridWorld.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import random

#############################################
# Introduction:
#
# This script demonstrates a simple reinforcement learning (RL)
# example using the Q-learning algorithm. The agent operates in a
# GridWorld environment (a 4×4 grid) where the goal is to move from
# the starting cell (top-left) to the goal cell (bottom-right).
#
# The Q-learning algorithm helps the agent learn the best action to
# take in each state by updating a Q-table using the Temporal Difference (TD)
# error. The agent uses an epsilon-greedy strategy to balance exploration
# (trying new actions) and exploitation (using the learned best actions).
#
# The code prints the evolving policy (represented with directional symbols)
# every 50 episodes so you can observe how the agent's policy adjusts as it
# learns. This code is designed to run in a Google Colab notebook.
#############################################

# Define the GridWorld environment class
class GridWorld:
    def __init__(self, size=4):
        self.size = size                      # Grid size (size x size grid)
        self.n_states = size * size           # Total number of states
        self.start_state = 0                  # Starting state (top-left corner)
        self.goal_state = self.n_states - 1   # Goal state (bottom-right corner)
        self.state = self.start_state         # Initialize current state

    def reset(self):
        """Reset the environment to the starting state."""
        self.state = self.start_state
        return self.state

    def step(self, action):
        """
        Executes the given action and returns the next state, reward, and done flag.

        Actions:
          0: up
          1: down
          2: left
          3: right

        The agent receives a small negative reward for each step to encourage
        finding the shortest path, and a positive reward upon reaching the goal.
        """
        # Convert current state into (row, col) coordinates
        row, col = divmod(self.state, self.size)

        # Update the row, col based on the chosen action with boundary conditions:
        if action == 0:      # Move up
            row = max(row - 1, 0)
        elif action == 1:    # Move down
            row = min(row + 1, self.size - 1)
        elif action == 2:    # Move left
            col = max(col - 1, 0)
        elif action == 3:    # Move right
            col = min(col + 1, self.size - 1)

        # Convert the new (row, col) back to a state index
        new_state = row * self.size + col

        # Determine the reward:
        #   +1 for reaching the goal, -0.1 for each step otherwise.
        reward = 1 if new_state == self.goal_state else -0.1

        # Check if the goal has been reached
        done = new_state == self.goal_state

        # Update the current state
        self.state = new_state

        return new_state, reward, done

#############################################
# Q-Learning Setup:
#
# We initialize a Q-table where each row corresponds to a state
# and each column corresponds to an action. The Q-values represent
# the expected reward for taking an action in a state.
#############################################

env = GridWorld(size=4)
n_actions = 4
q_table = np.zeros((env.n_states, n_actions))  # Initialize Q-table with zeros

# Q-learning hyperparameters
alpha = 0.1         # Learning rate: controls how much new information overrides old.
gamma = 0.99        # Discount factor: weighs future rewards relative to immediate ones.
epsilon = 1.0       # Exploration rate: probability of choosing a random action.
epsilon_decay = 0.995  # Decay rate for epsilon after each episode.
min_epsilon = 0.01  # Minimum value for epsilon to ensure some exploration.
num_episodes = 500  # Total number of episodes for training

def choose_action(state):
    """
    Epsilon-greedy strategy to choose an action:
      - With probability 'epsilon', choose a random action (exploration).
      - Otherwise, choose the action with the highest Q-value (exploitation).
    """
    if random.random() < epsilon:
        return random.randint(0, n_actions - 1)
    else:
        return np.argmax(q_table[state])

def print_policy():
    """
    Extracts the best action (policy) from the Q-table and prints it in a grid.

    The actions are mapped to symbols:
      0: up (^)
      1: down (v)
      2: left (<)
      3: right (>)
    """
    # Derive the policy by choosing the action with the highest Q-value at each state.
    policy = np.array([np.argmax(q_table[s]) for s in range(env.n_states)])
    symbols = {0: '^', 1: 'v', 2: '<', 3: '>'}  # Mapping from action to symbol
    policy_symbols = [symbols[a] for a in policy]

    # Print the policy grid row by row
    for i in range(env.size):
        print(policy_symbols[i*env.size:(i+1)*env.size])
    print()

#############################################
# Training Loop:
#
# For each episode, the agent starts at the beginning of the grid.
# It then chooses actions based on the current policy (with some exploration),
# updates the Q-table using the Q-learning update rule, and moves to the next state.
#
# The Q-learning update rule is:
#   Q(state, action) = Q(state, action) + alpha * [reward + gamma * max(Q(next_state)) - Q(state, action)]
#
# The policy is printed every 50 episodes to show how it evolves over time.
#############################################

for episode in range(1, num_episodes + 1):
    state = env.reset()  # Reset environment at the beginning of each episode
    done = False
    while not done:
        # Choose an action using epsilon-greedy strategy
        action = choose_action(state)

        # Execute the action and observe the next state, reward, and whether the goal is reached
        next_state, reward, done = env.step(action)

        # Identify the best action from the next state (for estimating the future reward)
        best_next_action = np.argmax(q_table[next_state])

        # Q-learning update:
        # Calculate the TD target (reward plus discounted estimated future reward)
        td_target = reward + gamma * q_table[next_state, best_next_action]

        # Calculate the TD error (difference between the target and current Q-value)
        td_error = td_target - q_table[state, action]

        # Update the Q-value for the current state and action
        q_table[state, action] += alpha * td_error

        # Transition to the next state
        state = next_state

    # Decay epsilon after each episode to reduce exploration over time
    epsilon = max(min_epsilon, epsilon * epsilon_decay)

    # Every 50 episodes, print the current policy so you can see the agent's progress
    if episode % 50 == 0:
        print(f"Episode {episode}:")
        print_policy()

#############################################
# Final Interpretation:
#
# After the training loop, the Q-table contains the estimated rewards for
# each state-action pair. The final policy (the best action for each state)
# is derived from the Q-table. This policy should represent an efficient
# path from the start state to the goal state.
#############################################

print("Final Q-Table:")
print(q_table)
print("Final policy:")
print_policy()


Episode 50:
['>', 'v', 'v', 'v']
['>', '>', '>', 'v']
['>', '>', 'v', 'v']
['>', '>', '>', '^']

Episode 100:
['>', 'v', 'v', 'v']
['>', '>', '>', 'v']
['>', 'v', 'v', 'v']
['>', '>', '>', '^']

Episode 150:
['>', 'v', 'v', 'v']
['>', '>', '>', 'v']
['>', 'v', 'v', 'v']
['>', '>', '>', '^']

Episode 200:
['>', 'v', 'v', 'v']
['>', '>', '>', 'v']
['>', 'v', 'v', 'v']
['>', '>', '>', '^']

Episode 250:
['>', 'v', 'v', 'v']
['>', '>', '>', 'v']
['>', 'v', 'v', 'v']
['>', '>', '>', '^']

Episode 300:
['>', 'v', 'v', 'v']
['>', '>', '>', 'v']
['>', 'v', 'v', 'v']
['>', '>', '>', '^']

Episode 350:
['>', 'v', 'v', 'v']
['>', '>', '>', 'v']
['>', 'v', 'v', 'v']
['>', '>', '>', '^']

Episode 400:
['>', 'v', 'v', 'v']
['>', '>', '>', 'v']
['>', 'v', 'v', 'v']
['>', '>', '>', '^']

Episode 450:
['>', 'v', 'v', 'v']
['>', '>', '>', 'v']
['>', 'v', 'v', 'v']
['>', '>', '>', '^']

Episode 500:
['>', 'v', 'v', 'v']
['>', '>', '>', 'v']
['>', 'v', 'v', 'v']
['>', '>', '>', '^']

Final Q-Table:
[[ 0.3